In [19]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-experimental neo4j

Note: you may need to restart the kernel to use updated packages.


In [20]:
import pickle
import os 
import numpy as np
import pandas as pd
import getpass
import os



input_file = "/home/jeans/nvaitc/battery_timeseries/resources/raw/test_bat_data.pkl"

with open(input_file, "rb") as file:
    test_dict = pickle.load(file)
    
for bat in test_dict:
    print(bat)
    trimmed_arr = np.trim_zeros(test_dict[bat]['q_d_n'], 'b')
    test_dict[bat]['trimmed_q_d_n'] = list(trimmed_arr)
    test_dict[bat]['gradient_q_d_n'] = list(np.gradient(trimmed_arr))
    trimmed_arr = trimmed_arr[len(trimmed_arr)//2:]
    print(np.gradient(trimmed_arr).shape)
    print(np.mean(np.gradient(trimmed_arr)))
    
    test_dict[bat]['mean_backhalf_grad'] = np.mean(np.gradient(trimmed_arr))

b3c0
(505,)
-0.0003375474769290131
b3c1
(532,)
-0.00031199782414543896
b3c3
(558,)
-0.00029250338513364075
b3c4
(525,)
-0.00032999958310808453
b3c5
(415,)
-0.00036687606788543336
b3c6
(334,)
-0.00034274847921497096
b3c7
(919,)
-0.00017601673180182688
b3c8
(415,)
-0.0004105239029390266
b3c9
(520,)
-0.00034236483849011935
b3c10
(540,)
-0.00030462763927601
b3c11
(409,)
-0.00041054688339419934
b3c12
(467,)
-0.0003878696317080512
b3c13
(409,)
-0.0004085456829490755
b3c14
(430,)
-0.0004199655943138655
b3c15
(439,)
-0.00038672131666561466
b3c16
(820,)
-0.00018805926892815567
b3c17
(658,)
-0.0002447868552976104
b3c18
(574,)
-0.00028483761727602226
b3c19
(578,)
-0.0002882764207450576
b3c20
(407,)
-0.0004205898512200583
b3c21
(387,)
-0.00034148609915445017
b3c22
(502,)
-0.0003200144881746209
b3c24
(413,)
-0.00037219247286891246
b3c25
(495,)
-0.000332798861493968
b3c26
(515,)
-0.0003389745082670045
b3c27
(426,)
-0.00038835532228711627
b3c28
(271,)
-0.0004227268079989951
b3c29
(430,)
-0.0003951689

In [21]:
sample_datapoint = test_dict[list(test_dict.keys())[0]]

mean_bh_grad = sample_datapoint['mean_backhalf_grad']
cycle = sample_datapoint['cycle']

print(mean_bh_grad, cycle)

-0.0003375474769290131 1010


In [22]:
os.environ["NEO4J_URI"] = "neo4j+s://3b31837b.databases.neo4j.io"
os.environ["NEO4J_USERNAME"] = "neo4j"
os.environ["NEO4J_PASSWORD"] = "D4W3Zfi44nAJfStBuxSE2DpKhlk_nMP6ybEjvOX5qxw"

from langchain_community.graphs import Neo4jGraph
graph = Neo4jGraph(refresh_schema=False)

graph.refresh_schema()
print(graph.schema)

Node properties:
Battery {batteryID: STRING, mean_backhalf_grad: FLOAT, degradation_cycle: INTEGER}
Cycle {batteryID: STRING, cycle_number: INTEGER, discharge_capacity: FLOAT, gradient: FLOAT}
Relationship properties:
NEXT_CYCLE {gradient: FLOAT}
The relationships:
(:Battery)-[:HAS_CYCLE]->(:Cycle)
(:Cycle)-[:NEXT_CYCLE]->(:Cycle)


In [ ]:
mean_bh_grad_range = 0.1
cycle_range = 50
#query db for similar node (:Battery) with similar mean_backhalf_grad and cycle
# aslo return new attributes showing how close the match is 
query = f"""
MATCH (b:Battery)
WHERE abs(b.mean_backhalf_grad - {mean_bh_grad}) < {mean_bh_grad_range}
AND abs(b.degradation_cycle - {cycle}) < {cycle_range}
RETURN b
"""
results = graph.query(query)    
for result in results:
    print(result['b'])
    print("mean_bh_grad_diff", abs(result['b']['mean_backhalf_grad'] - mean_bh_grad))
    print("cycle_diff", abs(result['b']['degradation_cycle'] - cycle))

{'batteryID': 'b1c24', 'mean_backhalf_grad': -0.0003455189918488088, 'degradation_cycle': 1018}
mean_bh_grad_diff 7.971514919795722e-06
cycle_diff 8
{'batteryID': 'b1c40', 'mean_backhalf_grad': -0.0003809121824939799, 'degradation_cycle': 965}
mean_bh_grad_diff 4.3364705564966804e-05
cycle_diff 45


In [ ]:
from langchain_nvidia_ai_endpoints import ChatNVIDIA

# Set up NVIDIA ChatNVIDIA LLM
llm = ChatNVIDIA(
    model="mistralai/mixtral-8x22b-instruct-v0.1",
    api_key=os.environ["NVIDIA_API_KEY"],
    temperature=0.5,
    top_p=1,
    max_tokens=1024,
)

# Query the graph database
query = """
MATCH (b:Battery)-[:HAS_CYCLE]->(c:Cycle)
RETURN b.batteryID AS batteryID, c.cycle_number AS cycle, c.discharge_capacity AS capacity
LIMIT 10
"""
result = graph.run(query)

# Format results for the LLM
data = [{"batteryID": record["batteryID"], "cycle": record["cycle"], "capacity": record["capacity"]} for record in result]

# Pass to LLM for further analysis
prompt = f"Analyze the following battery data and provide insights:\n{data}"
for chunk in llm.stream([{"role": "user", "content": prompt}]):
    print(chunk.content, end="")
